In [7]:
import pandas as pd
import math
import numpy as np
from geopy.distance import geodesic
from tqdm import tqdm
import random

# **Solution**

In [8]:
# Function to find the closest unvisited city using geodesic distance
def find_closest_city(data, start_coord, visited_cities):
    # Initialize variables to store the closest city
    min_distance = float('inf')
    closest_city = None
    closest_city_coord = None
    
    # Loop through each city to calculate the geodesic distance
    for index, row in data.iterrows():
        city_name = row[0]
        city_coord = (row[1], row[2])  
        
        # Skip the city if it's already visited
        if city_name in visited_cities:
            continue
        
        # Calculate geodesic distance between start_coord and city_coord
        distance = geodesic(start_coord, city_coord).kilometers
        
        # Update the closest city if this one is closer
        if distance < min_distance and distance != 0:
            min_distance = distance
            closest_city = city_name
            closest_city_coord = city_coord
    
    return closest_city, closest_city_coord, min_distance
    

# Function to solve the TSP using geodesic distance and nearest neighbor
def tsp_nearest_neighbor(data, start_city, start_city_coord):
    
    # Initialize visited cities array
    visited_cities = []
    
    # Add starting city to the visited list
    visited_cities.append(start_city)
    
    #print(f"Starting city: {start_city}")
    
    total_distance = 0
    current_city_coord = start_city_coord
    
    # Loop until all cities are visited
    with tqdm(total=len(data) - 1, desc="Visiting cities") as pbar:
        while len(visited_cities) < len(data):
            # Find the closest unvisited city
            closest_city, closest_city_coord, min_distance = find_closest_city(data, current_city_coord, visited_cities)
            
            # Add the closest city to the visited cities array
            visited_cities.append(closest_city)
            
            # Update the current city coordinates to the new closest city's coordinates
            current_city_coord = closest_city_coord
            
            # Accumulate the total distance
            total_distance += min_distance
            
            #print(f"Travels to {closest_city} with a distance of {min_distance:.2f} km")
            
            # Update the progress bar
            pbar.update(1)
    
    # Return to the start city to complete the loop
    return_to_start_distance = geodesic(current_city_coord, start_city_coord).kilometers
    total_distance += return_to_start_distance
    visited_cities.append(start_city)  # Add the starting city at the end to close the loop
    
    #print(f"Returning to {start_city} with a distance of {return_to_start_distance:.2f} km")
    
    return visited_cities, total_distance

In [9]:
# Import CSV file
def import_cities(filename):
    return pd.read_csv(filename)

# Choose a random starting city
def choose_random_city(data):
    random_index = np.random.choice(data.index)
    city_name = data.iloc[random_index][0]
    city_coord = (data.iloc[random_index][1], data.iloc[random_index][2])  
    return city_name, city_coord

In [11]:
#data = import_cities('vanuatu.csv')
data = import_cities('italy.csv')
#data = import_cities('us.csv')
#data = import_cities('russia.csv')
#data = import_cities('china.csv')
start_city, start_city_coord = choose_random_city(data)

/var/folders/pk/xy069ymn5cvb593dr0d4zmgh0000gn/T/ipykernel_36422/1238140414.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  city_name = data.iloc[random_index][0]
/var/folders/pk/xy069ymn5cvb593dr0d4zmgh0000gn/T/ipykernel_36422/1238140414.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  city_coord = (data.iloc[random_index][1], data.iloc[random_index][2])


In [12]:
# print of solution
visited_cities, total_distance = tsp_nearest_neighbor(data, start_city, start_city_coord)
print('------------------------------------------------------------')
print(f"Total distance traveled: {total_distance:.2f} km")
travel_order = " -> ".join(visited_cities)
print(f"Travel order: {travel_order}")

Visiting cities:   0%|          | 0/44 [00:00<?, ?it/s]/var/folders/pk/xy069ymn5cvb593dr0d4zmgh0000gn/T/ipykernel_36422/571662069.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  city_name = row[0]
/var/folders/pk/xy069ymn5cvb593dr0d4zmgh0000gn/T/ipykernel_36422/571662069.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  city_coord = (row[1], row[2])
Visiting cities: 100%|██████████| 44/44 [00:00<00:00, 196.02it/s]

------------------------------------------------------------
Total distance traveled: 5032.95 km
Travel order: Foggia -> Andria -> Bari -> Taranto -> Salerno -> Naples -> Giugliano in Campania -> Latina -> Rome -> Terni -> Perugia -> Rimini -> Forlì -> Ravenna -> Ferrara -> Bologna -> Modena -> Reggio nell'Emilia -> Parma -> Piacenza -> Milan -> Monza -> Bergamo -> Brescia -> Verona -> Vicenza -> Padua -> Venice -> Trieste -> Bolzano -> Trento -> Novara -> Turin -> Genoa -> Leghorn -> Prato -> Florence -> Pescara -> Palermo -> Catania -> Syracuse -> Reggio di Calabria -> Messina -> Cagliari -> Sassari -> Foggia
